In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import gridspec
import time
import tensorflow as tf
from sklearn.preprocessing import StandardScaler



import os
os.environ['CUDA_VISIBLE_DEVICES']="3" #"1"

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)



In [2]:
for obs in ['pT','qT','dphi','eta']:


    nominal_R = [np.load("storage_files/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4.npy")]
    #nominal_D += [np.load("storage_files/Django_nominaln_Omni_step2_"+obs+"_iteration4.npy")]
    print(obs,nominal_R[0], "\n")



pT [1.14153016e-01 4.42505935e-02 1.12746287e-02 2.48466471e-03
 4.61333135e-04 5.48850507e-05] 

qT [3.71788664 3.15515276 1.6217853  0.67385528 0.22026417 0.06857809
 0.00556623] 

dphi [6.0440479  3.61537102 2.04092379 1.01350616 0.42071553 0.15811563
 0.06210802] 

eta [0.30323704 0.58048175 0.34988284 0.13340729 0.0615625 ] 



In [3]:
bins = {}

#jet pt
bins[0] = np.logspace(np.log10(10),np.log10(100),7)

#jet eta
bins[1] = np.linspace(-1,2.5,6)

#dphi
bins[2] = np.logspace(np.log10(0.03),np.log10(np.pi/2.0),9) - 0.03
bins[2] = bins[2][1:]
bins[2][0] = 0.0

#qt
bins[3] = np.logspace(np.log10(0.03),np.log10(3.03),9) - 0.03
bins[3] = bins[3][1:]
bins[3][0] = 0.0

#Read in the data
data = pd.read_pickle("datafiles/data.pkl")
theta_unknown_S = data[['e_px','e_py','e_pz','jet_pt','jet_eta','jet_phi','jet_dphi','jet_qtnorm']].to_numpy()


scaler_data = StandardScaler()
scaler_data.fit(theta_unknown_S)

#Read in the MC
mc = pd.read_pickle("/clusterfs/ml4hep/bpnachman/H1/july16/datasets/Rapgap_nominal.pkl")
theta0_S = mc[['e_px','e_py','e_pz','jet_pt','jet_eta','jet_phi','jet_dphi','jet_qtnorm']].to_numpy()
theta0_G = mc[['gene_px','gene_py','gene_pz','genjet_pt','genjet_eta','genjet_phi','genjet_dphi','genjet_qtnorm']].to_numpy()
weights_MC_sim = mc['wgt']
pass_reco = np.array(mc['pass_reco'])
pass_truth = np.array(mc['pass_truth'])
pass_fiducial = np.array(mc['pass_fiducial'])

del mc
gc.collect()

22

In [4]:
NNweights_step2 = np.ones(len(theta0_S))
for i in range(5):
    mymodel = tf.keras.models.load_model("models/Rapgap_nominal_iteration"+str(i)+"_step2", compile=False)
    NNweights_step2_hold = mymodel.predict(scaler_data.transform(theta0_G),batch_size=10000)
    NNweights_step2_hold = NNweights_step2_hold/(1.-NNweights_step2_hold)
    NNweights_step2_hold = NNweights_step2_hold[:,0]
    NNweights_step2_hold = np.squeeze(np.nan_to_num(NNweights_step2_hold,posinf=1))
    NNweights_step2_hold[pass_truth==0] = 1.
    NNweights_step2 = NNweights_step2_hold*NNweights_step2

#trying to replicate result for qT

n_Omni_step2_qT,_,_=plt.hist(theta0_G[pass_fiducial==1][:,7],
                             bins=bins[3],weights=weights_MC_sim[pass_fiducial==1]*NNweights_step2[pass_fiducial==1],
                             density=True,histtype="step",color="black",ls=":",label="MC + step 2")

print("Replicated result with saved model", n_Omni_step2_qT) #could be the saved model is not just fro step 2...
print("Result from training session", np.load("storage_files/Rapgap_nominaln_Omni_step2_qT_iteration4.npy"))
print("Difference: ", n_Omni_step2_qT - np.load("storage_files/Rapgap_nominaln_Omni_step2_qT_iteration4.npy"))

Replicated result with saved model [3.71788664 3.15515276 1.6217853  0.67385528 0.22026417 0.06857809
 0.00556623]
Result from training session [3.71788664 3.15515276 1.6217853  0.67385528 0.22026417 0.06857809
 0.00556623]
Difference:  [0. 0. 0. 0. 0. 0. 0.]
